In [1]:
import pandas as pd
import xarray as xr

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure
from rad2bt import rad2bt

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
from readDMV import readDMV
c1 = readDMV('../data/paeri/190612C1.RNC')

In [4]:
noaa = pd.read_csv('../data/NOAA/noaamet/sum20190612_2359.csv', skiprows=0, parse_dates=True, index_col=0)

In [7]:
p1=figure(plot_width=900,
       plot_height=200,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='962-cm-1 microwindow radiance, 12 June 2019')
p1.line(c1.time.values,c1.mean_rad[:,957].values, color=Viridis3[0])

p2=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='Outside Temperatures, 12 June 2019')
p2.line(c1.time.values,c1.airNearBBsTemp.values, color=Viridis3[1], legend_label='airNearBBsTemp')
p2.line(c1.time.values,c1.outsideAirTemp.values, color=Viridis3[2], legend_label='outsideAirTemp')
p2.line(noaa.index,    noaa.iloc[:,13]+273.15,   color=Viridis3[0], legend_label='NOAA 2-m Temp')
p2.line([c1.time[0].values, c1.time[-1].values], [273.15, 273.15], color='blue', line_dash='dashed')

p3=figure(plot_width=900,
       plot_height=200,
       x_range=p1.x_range,
       x_axis_type='datetime',
       x_axis_label='Time (UTC)',
       y_axis_label='Radiance [mW m-2 sr-1 (cm-1)-1]',
       title='670-680 cm-1 Brightness Temperature, 12 June 2019')
p3.line(c1.time.values, rad2bt(c1.wnum1[374:395].mean(), c1.mean_rad[:,374:395].mean(axis=1)).values, color='darkblue')
p3.line([c1.time[0].values, c1.time[-1].values], [273.15, 273.15], color='blue', line_dash='dashed')

layout = gridplot([[p1], [p2], [p3]])
show(layout)

In [6]:
from glob import glob

from bokeh.palettes import Category20_11
from bokeh.layouts import row
#from bokeh.plotting import output_file

fns = glob('../data/radiosondes/190612/smtsondewnpnX1.b1.201906*.cdf')
fns.sort()

p1 = figure(plot_width=400, plot_height=600, y_range=[3, 13], title='Summit Radiosonde Profiles - June 2019 Melt Event')
for fn, color in zip(fns, Category20_11):
    ds  = xr.open_dataset(fn)
    snd = ds.to_dataframe()
    name = fn[-19:-4].replace('.', '_')
    p1.line(snd.tdry, snd.alt/1000, line_width=2, color=color, legend=name)
p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p2 = figure(plot_width=400, plot_height=600, y_range=p1.y_range)
for fn, color in zip(fns, Category20_11):
    ds  = xr.open_dataset(fn)
    snd = ds.to_dataframe()
    name = fn[-19:-4].replace('.', '_')
    p2.line(snd.rh, snd.alt/1000, line_width=2, color=color, legend=name)
p2.legend.location = "top_right"
p2.legend.click_policy="hide"

#output_file('/Users/vonw/work/projects/icecaps/analysis/Summit-Melt-Events/190612/SummitSondes_20190610-20190614.html')
show(row(p1, p2))

In [8]:
p4=figure(plot_width=900,
       plot_height=600,
       x_axis_label='Window Radiance [mW m-2 sr-1 (cm-1)-1]',
       y_axis_label='CO2 Brightness Temperature (K)',
       title='Window radiance vs. CO2 Brightness Temperature, 12 June 2019')
p4.scatter(c1.mean_rad[:,957].values, rad2bt(c1.wnum1[374:395].mean(), c1.mean_rad[:,374:395].mean(axis=1)).values, color=Viridis3[0])

show(p4)